In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
% cd /content

/content


In [ ]:
! git clone https://github.com/intel-isl/MiDaS.git

Cloning into 'MiDaS'...
remote: Enumerating objects: 250, done.
remote: Counting objects: 100% (250/250), done.
remote: Compressing objects: 100% (173/173), done.
remote: Total 394 (delta 58), reused 201 (delta 37), pack-reused 144
Receiving objects: 100% (394/394), 231.02 KiB | 9.63 MiB/s, done.
Resolving deltas: 100% (139/139), done.


In [ ]:
cd /content/MiDaS/

/content/MiDaS


In [ ]:
# ! git checkout -b old-state b00bf61f846d73fadc1f287293648db9f88d3615

In [ ]:
# ! cp /content/drive/My\ Drive/pre_trained/model-f46da743.pt /content/MiDaS/

In [ ]:
import os
import glob
import torch
import utils
import cv2

from torchvision.transforms import Compose
import torchvision.transforms as transforms
from midas.midas_net import MidasNet
from midas.transforms import Resize, NormalizeImage, PrepareForNet


In [ ]:
from torchsummary import summary

In [ ]:
import torch
import torch.nn as nn


class BaseModel(torch.nn.Module):
    def load(self, path):
        """Load model from file.

        Args:
            path (str): file path
        """
        parameters = torch.load(path, map_location=torch.device('cpu'))

        if "optimizer" in parameters:
            parameters = parameters["model"]

        self.load_state_dict(parameters)


def _make_encoder(backbone, features, use_pretrained, groups=1, expand=False, exportable=True):
    if backbone == "resnext101_wsl":
        pretrained = _make_pretrained_resnext101_wsl(use_pretrained)
        scratch = _make_scratch([256, 512, 1024, 2048], features, groups=groups, expand=expand)     # efficientnet_lite3  
    elif backbone == "efficientnet_lite3":
        pretrained = _make_pretrained_efficientnet_lite3(use_pretrained, exportable=exportable)
        scratch = _make_scratch([32, 48, 136, 384], features, groups=groups, expand=expand)  # efficientnet_lite3     
    else:
        print(f"Backbone '{backbone}' not implemented")
        assert False
        
    return pretrained, scratch


def _make_scratch(in_shape, out_shape, groups=1, expand=False):
    scratch = nn.Module()

    out_shape1 = out_shape
    out_shape2 = out_shape
    out_shape3 = out_shape
    out_shape4 = out_shape
    if expand==True:
        out_shape1 = out_shape
        out_shape2 = out_shape*2
        out_shape3 = out_shape*4
        out_shape4 = out_shape*8

    scratch.layer1_rn = nn.Conv2d(
        in_shape[0], out_shape1, kernel_size=3, stride=1, padding=1, bias=False, groups=groups
    )
    scratch.layer2_rn = nn.Conv2d(
        in_shape[1], out_shape2, kernel_size=3, stride=1, padding=1, bias=False, groups=groups
    )
    scratch.layer3_rn = nn.Conv2d(
        in_shape[2], out_shape3, kernel_size=3, stride=1, padding=1, bias=False, groups=groups
    )
    scratch.layer4_rn = nn.Conv2d(
        in_shape[3], out_shape4, kernel_size=3, stride=1, padding=1, bias=False, groups=groups
    )

    return scratch


def _make_pretrained_efficientnet_lite3(use_pretrained, exportable=False):
    efficientnet = torch.hub.load(
        "rwightman/gen-efficientnet-pytorch",
        "tf_efficientnet_lite3",
        pretrained=use_pretrained,
        exportable=exportable
    )
    return _make_efficientnet_backbone(efficientnet)


def _make_efficientnet_backbone(effnet):
    pretrained = nn.Module()

    pretrained.layer1 = nn.Sequential(
        effnet.conv_stem, effnet.bn1, effnet.act1, *effnet.blocks[0:2]
    )
    pretrained.layer2 = nn.Sequential(*effnet.blocks[2:3])
    pretrained.layer3 = nn.Sequential(*effnet.blocks[3:5])
    pretrained.layer4 = nn.Sequential(*effnet.blocks[5:9])

    return pretrained
    

def _make_resnet_backbone(resnet):
    pretrained = nn.Module()
    pretrained.layer1 = nn.Sequential(
        resnet.conv1, resnet.bn1, resnet.relu, resnet.maxpool, resnet.layer1
    )

    pretrained.layer2 = resnet.layer2
    pretrained.layer3 = resnet.layer3
    pretrained.layer4 = resnet.layer4

    return pretrained


def _make_pretrained_resnext101_wsl(use_pretrained):
    resnet = torch.hub.load("facebookresearch/WSL-Images", "resnext101_32x8d_wsl")
    # print(resnet)
    return _make_resnet_backbone(resnet)



class Interpolate(nn.Module):
    """Interpolation module.
    """

    def __init__(self, scale_factor, mode):
        """Init.

        Args:
            scale_factor (float): scaling
            mode (str): interpolation mode
        """
        super(Interpolate, self).__init__()

        self.interp = nn.functional.interpolate
        self.scale_factor = scale_factor
        self.mode = mode

    def forward(self, x):
        """Forward pass.

        Args:
            x (tensor): input

        Returns:
            tensor: interpolated data
        """

        x = self.interp(
            x, scale_factor=self.scale_factor, mode=self.mode, align_corners=False
        )

        return x


class ResidualConvUnit(nn.Module):
    """Residual convolution module.
    """

    def __init__(self, features):
        """Init.

        Args:
            features (int): number of features
        """
        super().__init__()

        self.conv1 = nn.Conv2d(
            features, features, kernel_size=3, stride=1, padding=1, bias=True
        )

        self.conv2 = nn.Conv2d(
            features, features, kernel_size=3, stride=1, padding=1, bias=True
        )

        self.relu = nn.ReLU(inplace=True)

    def forward(self, x):
        """Forward pass.

        Args:
            x (tensor): input

        Returns:
            tensor: output
        """
        out = self.relu(x)
        out = self.conv1(out)
        out = self.relu(out)
        out = self.conv2(out)

        return out + x


class FeatureFusionBlock(nn.Module):
    """Feature fusion block.
    """

    def __init__(self, features):
        """Init.

        Args:
            features (int): number of features
        """
        super(FeatureFusionBlock, self).__init__()

        self.resConfUnit1 = ResidualConvUnit(features)
        self.resConfUnit2 = ResidualConvUnit(features)

    def forward(self, *xs):
        """Forward pass.

        Returns:
            tensor: output
        """
        output = xs[0]

        if len(xs) == 2:
            output += self.resConfUnit1(xs[1])

        output = self.resConfUnit2(output)

        output = nn.functional.interpolate(
            output, scale_factor=2, mode="bilinear", align_corners=True
        )

        return output




class ResidualConvUnit_custom(nn.Module):
    """Residual convolution module.
    """

    def __init__(self, features, activation, bn):
        """Init.

        Args:
            features (int): number of features
        """
        super().__init__()

        self.bn = bn

        self.groups=1

        self.conv1 = nn.Conv2d(
            features, features, kernel_size=3, stride=1, padding=1, bias=True, groups=self.groups
        )
        
        self.conv2 = nn.Conv2d(
            features, features, kernel_size=3, stride=1, padding=1, bias=True, groups=self.groups
        )

        if self.bn==True:
            self.bn1 = nn.BatchNorm2d(features)
            self.bn2 = nn.BatchNorm2d(features)

        self.activation = activation

        self.skip_add = nn.quantized.FloatFunctional()

    def forward(self, x):
        """Forward pass.

        Args:
            x (tensor): input

        Returns:
            tensor: output
        """
        
        out = self.activation(x)
        out = self.conv1(out)
        if self.bn==True:
            out = self.bn1(out)
       
        out = self.activation(out)
        out = self.conv2(out)
        if self.bn==True:
            out = self.bn2(out)

        if self.groups > 1:
            out = self.conv_merge(out)

        return self.skip_add.add(out, x)

        # return out + x


class FeatureFusionBlock_custom(nn.Module):
    """Feature fusion block.
    """

    def __init__(self, features, activation, deconv=False, bn=False, expand=False, align_corners=True):
        """Init.

        Args:
            features (int): number of features
        """
        super(FeatureFusionBlock_custom, self).__init__()

        self.deconv = deconv
        self.align_corners = align_corners

        self.groups=1

        self.expand = expand
        out_features = features
        if self.expand==True:
            out_features = features//2
        
        self.out_conv = nn.Conv2d(features, out_features, kernel_size=1, stride=1, padding=0, bias=True, groups=1)

        self.resConfUnit1 = ResidualConvUnit_custom(features, activation, bn)
        self.resConfUnit2 = ResidualConvUnit_custom(features, activation, bn)
        
        self.skip_add = nn.quantized.FloatFunctional()

    def forward(self, *xs):
        """Forward pass.

        Returns:
            tensor: output
        """
        output = xs[0]

        if len(xs) == 2:
            res = self.resConfUnit1(xs[1])
            output = self.skip_add.add(output, res)
            # output += res

        output = self.resConfUnit2(output)

        output = nn.functional.interpolate(
            output, scale_factor=2, mode="bilinear", align_corners=self.align_corners
        )

        output = self.out_conv(output)

        return output



In [ ]:

import torch
import torch.nn as nn

class MidasNet(BaseModel):
    """Network for monocular depth estimation.
    """

    def __init__(self, path=None, features=256, non_negative=True):
        """Init.

        Args:
            path (str, optional): Path to saved model. Defaults to None.
            features (int, optional): Number of features. Defaults to 256.
            backbone (str, optional): Backbone network for encoder. Defaults to resnet50
        """
        print("Loading weights: ", path)

        super(MidasNet, self).__init__()

        use_pretrained = False if path is None else True

        self.pretrained, self.scratch = _make_encoder(backbone="resnext101_wsl", features=features, use_pretrained=use_pretrained, expand = True)

        self.scratch.refinenet4 = FeatureFusionBlock(features)
        self.scratch.refinenet3 = FeatureFusionBlock(features)
        self.scratch.refinenet2 = FeatureFusionBlock(features)
        self.scratch.refinenet1 = FeatureFusionBlock(features)

        self.scratch.output_conv = nn.Sequential(
            nn.Conv2d(features, 128, kernel_size=3, stride=1, padding=1),
            Interpolate(scale_factor=2, mode="bilinear"),
            nn.Conv2d(128, 32, kernel_size=3, stride=1, padding=1),
            nn.ReLU(True),
            nn.Conv2d(32, 1, kernel_size=1, stride=1, padding=0),
            nn.ReLU(True) if non_negative else nn.Identity(),
        )

        if path:
            self.load(path)

    def forward(self, x):
        """Forward pass.

        Args:
            x (tensor): input data (image)

        Returns:
            tensor: depth
        """

        layer_1 = self.pretrained.layer1(x)
        layer_2 = self.pretrained.layer2(layer_1)
        layer_3 = self.pretrained.layer3(layer_2)
        layer_4 = self.pretrained.layer4(layer_3)
        # print(layer_2.size())
        # print(layer_3.size())
        # print(layer_4.size())
        layer_1_rn = self.scratch.layer1_rn(layer_1)
        layer_2_rn = self.scratch.layer2_rn(layer_2)
        layer_3_rn = self.scratch.layer3_rn(layer_3)
        layer_4_rn = self.scratch.layer4_rn(layer_4)
        # print(layer_2_rn.size())
        # print(layer_3_rn.size())
        # print(layer_4_rn.size())
        path_4 = self.scratch.refinenet4(layer_4_rn)
        path_3 = self.scratch.refinenet3(path_4, layer_3_rn)
        path_2 = self.scratch.refinenet2(path_3, layer_2_rn)
        path_1 = self.scratch.refinenet1(path_2, layer_1_rn)

        out = self.scratch.output_conv(path_1)

        return torch.squeeze(out, dim=1)


In [ ]:
input_path = "ConstructionPPE/input_images"
output_path = "output"
# MODEL_PATH = "model.pt"
model_path = None

In [ ]:
model = MidasNet(model_path, non_negative=True).to('cuda' if torch.cuda.is_available() else 'cpu')

Loading weights:  None


Using cache found in /root/.cache/torch/hub/facebookresearch_WSL-Images_master


In [ ]:
# print(model.pretrained.layer1, (3, 416, 416))

In [ ]:
# print(summary(model.pretrained.layer1, (3, 416, 416)))

In [ ]:
# print(summary(model.pretrained.layer2, (256, 104, 104)))

In [ ]:
# print(summary(model.pretrained.layer3, (512, 52, 52)))

In [ ]:
# print(summary(model.pretrained.layer4, (1024, 26, 26)))

In [ ]:
# print(model.scratch.layer1_rn)

In [ ]:
# print(model.scratch.layer2_rn)

In [ ]:
# print(model.scratch.layer3_rn)

In [ ]:
# print(model.scratch.layer4_rn)

In [ ]:
import torch
import torch.nn as nn
from collections import OrderedDict


class YOLO(nn.Module):
    def __init__(self, config, backbone, neck, is_training=True):
        super(YOLO, self).__init__()
        self.config = config
        self.training = is_training

        self.backbone = backbone
        # self.neck = neck
        _out_filters = [256, 512, 1024, 2048]
        # _out_filters = [128, 256, 512, 1024]
        #  embedding0
        final_out_filter0 = len(config["yolo"]["anchors"][0]) * (5 + config["yolo"]["classes"])
        self.embedding0 = self._make_embedding([512, 2048], _out_filters[-1], final_out_filter0)
        #  embedding1
        final_out_filter1 = len(config["yolo"]["anchors"][1]) * (5 + config["yolo"]["classes"])
        self.embedding1_cbl = self._make_cbl(512, 256, 1)
        self.embedding1_upsample = nn.Upsample(scale_factor=2, mode='nearest')
        self.embedding1 = self._make_embedding([256, 1024], _out_filters[-2] + 256, final_out_filter1)
        #  embedding2
        final_out_filter2 = len(config["yolo"]["anchors"][2]) * (5 + config["yolo"]["classes"])
        self.embedding2_cbl = self._make_cbl(256, 128, 1)
        self.embedding2_upsample = nn.Upsample(scale_factor=2, mode='nearest')
        self.embedding2 = self._make_embedding([128, 512], _out_filters[-3] + 128, final_out_filter2)

    def _make_cbl(self, _in, _out, ks):
        pad = (ks - 1) // 2 if ks else 0
        return nn.Sequential(OrderedDict([
            ("conv", nn.Conv2d(_in, _out, kernel_size=ks, stride=1, padding=pad, bias=False)),
            ("bn", nn.BatchNorm2d(_out)),
            ("relu", nn.LeakyReLU(0.1)),
        ]))

    def _make_embedding(self, filters_list, in_filters, out_filter):
        m = nn.ModuleList([
            self._make_cbl(in_filters, filters_list[0], 1),
            self._make_cbl(filters_list[0], filters_list[1], 3),
            self._make_cbl(filters_list[1], filters_list[0], 1),
            self._make_cbl(filters_list[0], filters_list[1], 3),
            self._make_cbl(filters_list[1], filters_list[0], 1),
            self._make_cbl(filters_list[0], filters_list[1], 3)])
        m.add_module("conv_out", nn.Conv2d(filters_list[1], out_filter, kernel_size=1,
                                           stride=1, padding=0, bias=True))
        return m

    def forward(self, x):

        layer_1 = self.backbone.layer1(x)
        x2 = self.backbone.layer2(layer_1)
        x1 = self.backbone.layer3(x2)
        x0 = self.backbone.layer4(x1)
        # print(x2.size() , x1.size(), x0.size())

        # x2, x1, x0 = torch.split(x2, 256, dim = 1), torch.split(x1, 512, dim = 1), torch.split(x0, 1024, dim = 1)
        # x2, x1, x0 = torch.cat(x2, dim = 0), torch.cat(x1, dim = 0), torch.cat(x0, dim = 0)
        # print(x2.size() , x1.size(), x0.size())
        # x = self.neck.layer1_rn(layer_1)
        # x2 = self.neck.layer2_rn(layer_2)
        # x1 = self.neck.layer3_rn(layer_3)
        # x0 = self.neck.layer4_rn(layer_4)
        # print(x2.size())
        # print(x1.size())
        # print(x0.size())
        
        def _branch(_embedding, _in):
            for i, e in enumerate(_embedding):
                _in = e(_in)
                if i == 4:
                    out_branch = _in
            return _in, out_branch
        #  backbone
        # x2, x1, x0 = self.backbone(x)

        # x2, x1, x0 = self.backbone.layer2, self.backbone.layer3, self.backbone.layer4
        
        #  yolo branch 0
        out0, out0_branch = _branch(self.embedding0, x0)
        #  yolo branch 1
        x1_in = self.embedding1_cbl(out0_branch)
        x1_in = self.embedding1_upsample(x1_in)
        x1_in = torch.cat([x1_in, x1], 1)
        out1, out1_branch = _branch(self.embedding1, x1_in)
        #  yolo branch 2
        x2_in = self.embedding2_cbl(out1_branch)
        x2_in = self.embedding2_upsample(x2_in)
        x2_in = torch.cat([x2_in, x2], 1)
        out2, out2_branch = _branch(self.embedding2, x2_in)
        return out0, out1, out2

In [ ]:
# _out_filters = [256, 512, 1024, 2048]

In [ ]:
config = {"yolo": {
    "anchors": [[[116, 90], [156, 198], [373, 326]],
                [[30, 61], [62, 45], [59, 119]],
                [[10, 13], [16, 30], [33, 23]]],
    "classes": 4,
}
,
"lr": {
        "backbone_lr": 0.001,
        "other_lr": 0.01,
        "freeze_backbone": True,   #  freeze backbone wegiths to finetune
        "decay_gamma": 0.1,
        "decay_step": 20,           #  decay lr in every ? epochs
    },
    "optimizer": {
        "type": "sgd",
        "weight_decay": 4e-05,
    },
    "batch_size": 4,
    "train_path": "../data/coco/trainvalno5k.txt",
    "epochs": 100,
    "img_h": 416,
    "img_w": 416,
    "parallels": [0],                         #  config GPU device
    "working_dir": "YOUR_WORKING_DIR",              #  replace with your working dir
    "pretrain_snapshot": "",                        #  load checkpoint
    "evaluate_type": "", 
    "try": 0,
    "export_onnx": False,

}

In [ ]:
detector = YOLO(config, model.pretrained, model.scratch, False).to('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
# print(detector.modules)

In [ ]:
# print(summary(detector, (3, 416, 416)))

In [ ]:
! gdown --id 1SnFAlSvsx37J7MDNs3WWLgeKY0iknikP

Downloading...
From: https://drive.google.com/uc?id=1SnFAlSvsx37J7MDNs3WWLgeKY0iknikP
To: /content/MiDaS/official_yolov3_weights_pytorch.pth
248MB [00:01, 163MB/s]


In [ ]:
# print(detector)

In [ ]:
# for name, param in detector.embedding0.named_parameters():
#     if param.requires_grad:
#         print (name, param.data)

In [ ]:
# detector = nn.DataParallel(detector)

In [ ]:
state_dict = torch.load('official_yolov3_weights_pytorch.pth', map_location = torch.device('cuda' if torch.cuda.is_available() else 'cpu'))

In [ ]:

detector.load_state_dict(state_dict, strict = False)

_IncompatibleKeys(missing_keys=['backbone.layer1.0.weight', 'backbone.layer1.1.weight', 'backbone.layer1.1.bias', 'backbone.layer1.1.running_mean', 'backbone.layer1.1.running_var', 'backbone.layer1.4.0.conv1.weight', 'backbone.layer1.4.0.bn1.weight', 'backbone.layer1.4.0.bn1.bias', 'backbone.layer1.4.0.bn1.running_mean', 'backbone.layer1.4.0.bn1.running_var', 'backbone.layer1.4.0.conv2.weight', 'backbone.layer1.4.0.bn2.weight', 'backbone.layer1.4.0.bn2.bias', 'backbone.layer1.4.0.bn2.running_mean', 'backbone.layer1.4.0.bn2.running_var', 'backbone.layer1.4.0.conv3.weight', 'backbone.layer1.4.0.bn3.weight', 'backbone.layer1.4.0.bn3.bias', 'backbone.layer1.4.0.bn3.running_mean', 'backbone.layer1.4.0.bn3.running_var', 'backbone.layer1.4.0.downsample.0.weight', 'backbone.layer1.4.0.downsample.1.weight', 'backbone.layer1.4.0.downsample.1.bias', 'backbone.layer1.4.0.downsample.1.running_mean', 'backbone.layer1.4.0.downsample.1.running_var', 'backbone.layer1.4.1.conv1.weight', 'backbone.layer1

In [ ]:
# ! pip3 install torchviz

In [ ]:
% cd /content/MiDaS/

/content/MiDaS


In [ ]:
# ! git clone https://github.com/BobLiu20/YOLOv3_PyTorch.git

In [ ]:
% cd YOLOv3_PyTorch/

/content/MiDaS/YOLOv3_PyTorch


In [ ]:
from nets.yolo_loss import YOLOLoss
from common.coco_dataset import COCODataset

In [ ]:
yolo_losses = []
for i in range(3):
    yolo_losses.append(YOLOLoss(config["yolo"]["anchors"][i],
                                config["yolo"]["classes"], (416, 416)))

In [ ]:
% cd /content/
! git clone https://github.com/theschoolofai/YoloV3.git
% cd /content/MiDaS/YOLOv3_PyTorch/

/content
fatal: destination path 'YoloV3' already exists and is not an empty directory.
/content/MiDaS/YOLOv3_PyTorch


In [ ]:
# ! cp -r /content/YoloV3/data/smalcoco/ /content/MiDaS/YOLOv3_PyTorch/data/

In [ ]:
% cd /content/MiDaS/YOLOv3_PyTorch/

/content/MiDaS/YOLOv3_PyTorch


In [ ]:
# DataLoader
dataloader = torch.utils.data.DataLoader(COCODataset("/content/MiDaS/YOLOv3_PyTorch/data/custom/input_file_dir.txt",
                                                        (416, 416),
                                                        is_training=True),
                                            batch_size=4,
                                            shuffle=True, num_workers=16, pin_memory=True)

In [ ]:
# ! cp -r /content/drive/MyDrive/yolo_data /content/MiDaS/YOLOv3_PyTorch/data/

In [ ]:
# dataloader = torch.utils.data.DataLoader(LoadImagesAndLabels("./data/yolo_data/customdata/train.txt"))

In [ ]:
sample = next(iter(dataloader))

In [ ]:
# print(sample[0].shape)
# print(sample[1].shape)

In [ ]:
import logging
import torch.optim as optim

In [ ]:
import time
def _get_optimizer(config, net):
    optimizer = None

    # Assign different lr for each layer
    params = None
    base_params = list(
        map(id, net.backbone.parameters())
    )
    logits_params = filter(lambda p: id(p) not in base_params, net.parameters())

    if not config["lr"]["freeze_backbone"]:
        params = [
            {"params": logits_params, "lr": config["lr"]["other_lr"]},
            {"params": net.backbone.parameters(), "lr": config["lr"]["backbone_lr"]},
        ]
    else:
        logging.info("freeze backbone's parameters.")
        for p in net.backbone.parameters():
            p.requires_grad = False
        params = [
            {"params": logits_params, "lr": config["lr"]["other_lr"]},
        ]

    # Initialize optimizer class
    if config["optimizer"]["type"] == "adam":
        optimizer = optim.Adam(params, weight_decay=config["optimizer"]["weight_decay"])
    elif config["optimizer"]["type"] == "amsgrad":
        optimizer = optim.Adam(params, weight_decay=config["optimizer"]["weight_decay"],
                               amsgrad=True)
    elif config["optimizer"]["type"] == "rmsprop":
        optimizer = optim.RMSprop(params, weight_decay=config["optimizer"]["weight_decay"])
    else:
        # Default to sgd
        logging.info("Using SGD optimizer.")
        optimizer = optim.SGD(params, momentum=0.9,
                              weight_decay=config["optimizer"]["weight_decay"],
                              nesterov=(config["optimizer"]["type"] == "nesterov"))

    return optimizer


In [ ]:
optimizer = _get_optimizer(config, detector)
lr_scheduler = optim.lr_scheduler.StepLR(
    optimizer,
    step_size=config["lr"]["decay_step"],
    gamma=config["lr"]["decay_gamma"])

In [ ]:
def _save_checkpoint(state_dict):
    checkpoint_path = "/content/MiDaS/saved_model.pth"
    torch.save(state_dict, checkpoint_path)

In [ ]:
for epoch in range(80):
    print('epoch', epoch)
    total_loss = 0
    for step, samples in enumerate(dataloader):
        # sample = samples[0][0]
        # images, labels = samples[0][0], samples[1][0]
        # images, labels = samples[0], samples[1]
        images, labels = samples['image'], samples['label']
        # print(labels)
        device = 'cuda' if torch.cuda.is_available() else 'cpu'
        images, lables = images.to(device, dtype = torch.float), labels.to(device, dtype = torch.float)
        start_time = time.time()
        # config["global_step"] += 1

        # Forward and backward
        optimizer.zero_grad()
        outputs = detector(images)
        # print(len(images))
        # print(images.shape)
        # print(len(outputs))
        # print(outputs[0].shape)
        # print(samples['image_path'])
        losses_name = ["total_loss", "x", "y", "w", "h", "conf", "cls"]
        losses = []
        for _ in range(len(losses_name)):
            losses.append([])
        for i in range(3):
            _loss_item = yolo_losses[i](outputs[i], labels)
            for j, l in enumerate(_loss_item):
                losses[j].append(l)
        losses = [sum(l) for l in losses]
        loss = losses[0]
        loss.backward()
        # print(loss)
        total_loss += loss
        optimizer.step()
        # print(loss)
        if step > 0 and step % 10 == 0:
            _loss = loss.item()
            duration = float(time.time() - start_time)
            example_per_second = config["batch_size"] / duration
            lr = optimizer.param_groups[0]['lr']
            print(epoch, step, _loss, example_per_second, lr)
            logging.info(
                "epoch [%.3d] iter = %d loss = %.2f example/sec = %.3f lr = %.5f "%
                (epoch, step, _loss, example_per_second, lr)
            )
            # config["tensorboard_writer"].add_scalar("lr",
            #                                         lr,
            #                                         config["global_step"])
            # config["tensorboard_writer"].add_scalar("example/sec",
            #                                         example_per_second,
            #                                         config["global_step"])
            for i, name in enumerate(losses_name):
                value = _loss if i == 0 else losses[i]
                # config["tensorboard_writer"].add_scalar(name,
                #                                         value,
                #                                         config["global_step"])

        if step > 0 and step % 860 == 0:
            detector.train(False)
            _save_checkpoint(detector.state_dict())
            detector.train(True)

    lr_scheduler.step()
    print(total_loss/4)
detector.train(False)
_save_checkpoint(detector.state_dict())
detector.train(True)
logging.info("Bye~")

epoch 0
0 10 0.24229156970977783 13.3806673897786 0.01
0 20 1.0431218147277832 12.762366830596727 0.01
0 30 0.28811752796173096 12.862875705448003 0.01
0 40 0.2521117329597473 12.774474830946161 0.01
0 50 0.15034037828445435 12.030119008949514 0.01
0 60 0.4553418755531311 10.828565306180407 0.01
0 70 1.272276759147644 11.861308537812251 0.01
0 80 0.21085157990455627 11.970412905076284 0.01
0 90 0.5299996137619019 11.023833303546084 0.01
0 100 0.5856153964996338 10.051449434257526 0.01
0 110 2.286013126373291 11.619592581896054 0.01
0 120 0.915529727935791 12.055282426402432 0.01
0 130 0.6528151631355286 11.654834606570166 0.01
0 140 0.5314655303955078 10.880293025469818 0.01
0 150 0.3455304503440857 12.153818402833368 0.01
0 160 0.2979307770729065 11.716513877433867 0.01
0 170 0.8336289525032043 10.169414925213909 0.01
0 180 0.4534981846809387 12.003875076020464 0.01
0 190 0.6094105243682861 12.875817535199486 0.01
0 200 0.32847514748573303 12.607130297301115 0.01
0 210 0.6336209177970

In [ ]:
['./data/custom/images/image_3027.jpg', './data/custom/images/image_0522.jpg', './data/custom/images/image_1944.jpg', './data/custom/images/image_2820.jpg']

['./data/custom/images/image_3027.jpg',
 './data/custom/images/image_0522.jpg',
 './data/custom/images/image_1944.jpg',
 './data/custom/images/image_2820.jpg']

In [ ]:
import numpy as np
import random

import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from matplotlib.ticker import NullLocator

In [ ]:
cmap = plt.get_cmap('tab20b')
colors = [cmap(i) for i in np.linspace(0, 1, 5)]
print(len(colors))

In [ ]:
def non_max_suppression(prediction, num_classes, conf_thres=0.5, nms_thres=0.4):
    """
    Removes detections with lower object confidence score than 'conf_thres' and performs
    Non-Maximum Suppression to further filter detections.
    Returns detections with shape:
        (x1, y1, x2, y2, object_conf, class_score, class_pred)
    """

    # From (center x, center y, width, height) to (x1, y1, x2, y2)
    box_corner = prediction.new(prediction.shape)
    box_corner[:, :, 0] = prediction[:, :, 0] - prediction[:, :, 2] / 2
    box_corner[:, :, 1] = prediction[:, :, 1] - prediction[:, :, 3] / 2
    box_corner[:, :, 2] = prediction[:, :, 0] + prediction[:, :, 2] / 2
    box_corner[:, :, 3] = prediction[:, :, 1] + prediction[:, :, 3] / 2
    prediction[:, :, :4] = box_corner[:, :, :4]

    output = [None for _ in range(len(prediction))]
    for image_i, image_pred in enumerate(prediction):
        # Filter out confidence scores below threshold
        conf_mask = (image_pred[:, 4] >= conf_thres).squeeze()
        image_pred = image_pred[conf_mask]
        # If none are remaining => process next image
        if not image_pred.size(0):
            continue
        # Get score and class with highest confidence
        class_conf, class_pred = torch.max(image_pred[:, 5:5 + num_classes], 1,  keepdim=True)
        # Detections ordered as (x1, y1, x2, y2, obj_conf, class_conf, class_pred)
        detections = torch.cat((image_pred[:, :5], class_conf.float(), class_pred.float()), 1)
        # Iterate through all predicted classes
        unique_labels = detections[:, -1].cpu().unique()
        if prediction.is_cuda:
            unique_labels = unique_labels.cuda()
        for c in unique_labels:
            # Get the detections with the particular class
            detections_class = detections[detections[:, -1] == c]
            # Sort the detections by maximum objectness confidence
            _, conf_sort_index = torch.sort(detections_class[:, 4], descending=True)
            detections_class = detections_class[conf_sort_index]
            # Perform non-maximum suppression
            max_detections = []
            while detections_class.size(0):
                # Get detection with highest confidence and save as max detection
                max_detections.append(detections_class[0].unsqueeze(0))
                # Stop if we're at the last detection
                if len(detections_class) == 1:
                    break
                # Get the IOUs for all boxes with lower confidence
                ious = bbox_iou(max_detections[-1], detections_class[1:])
                # Remove detections with IoU >= NMS threshold
                detections_class = detections_class[1:][ious < nms_thres]

            max_detections = torch.cat(max_detections).data
            # Add max detections to outputs
            output[image_i] = max_detections if output[image_i] is None else torch.cat((output[image_i], max_detections))

    return output

def bbox_iou(box1, box2, x1y1x2y2=True):
    """
    Returns the IoU of two bounding boxes
    """
    if not x1y1x2y2:
        # Transform from center and width to exact coordinates
        b1_x1, b1_x2 = box1[:, 0] - box1[:, 2] / 2, box1[:, 0] + box1[:, 2] / 2
        b1_y1, b1_y2 = box1[:, 1] - box1[:, 3] / 2, box1[:, 1] + box1[:, 3] / 2
        b2_x1, b2_x2 = box2[:, 0] - box2[:, 2] / 2, box2[:, 0] + box2[:, 2] / 2
        b2_y1, b2_y2 = box2[:, 1] - box2[:, 3] / 2, box2[:, 1] + box2[:, 3] / 2
    else:
        # Get the coordinates of bounding boxes
        b1_x1, b1_y1, b1_x2, b1_y2 = box1[:,0], box1[:,1], box1[:,2], box1[:,3]
        b2_x1, b2_y1, b2_x2, b2_y2 = box2[:,0], box2[:,1], box2[:,2], box2[:,3]

    # get the corrdinates of the intersection rectangle
    inter_rect_x1 =  torch.max(b1_x1, b2_x1)
    inter_rect_y1 =  torch.max(b1_y1, b2_y1)
    inter_rect_x2 =  torch.min(b1_x2, b2_x2)
    inter_rect_y2 =  torch.min(b1_y2, b2_y2)
    # Intersection area
    inter_area =    torch.clamp(inter_rect_x2 - inter_rect_x1 + 1, min=0) * \
                    torch.clamp(inter_rect_y2 - inter_rect_y1 + 1, min=0)
    # Union Area
    b1_area = (b1_x2 - b1_x1 + 1) * (b1_y2 - b1_y1 + 1)
    b2_area = (b2_x2 - b2_x1 + 1) * (b2_y2 - b2_y1 + 1)

    iou = inter_area / (b1_area + b2_area - inter_area + 1e-16)

    return iou

In [ ]:
# YOLO loss with 3 scales
yolo_losses = []
for i in range(3):
    yolo_losses.append(YOLOLoss(config["yolo"]["anchors"][i],
                                config["yolo"]["classes"], (config["img_w"], config["img_h"])))

# prepare images path
images_name = os.listdir('/content/MiDaS/YOLOv3_PyTorch/data/yolo_data/customdata/images/')
images_path = [os.path.join('/content/MiDaS/YOLOv3_PyTorch/data/yolo_data/customdata/images/', name) for name in images_name]
if len(images_path) == 0:
    raise Exception("no image found in {}".format(config["images_path"]))

# Start inference
batch_size = config["batch_size"]
for step in range(0, len(images_path), batch_size):
    # preprocess
    images = []
    images_origin = []
    for path in images_path[step*batch_size: (step+1)*batch_size]:
        logging.info("processing: {}".format(path))
        image = cv2.imread(path, cv2.IMREAD_COLOR)
        if image is None:
            logging.error("read path error: {}. skip it.".format(path))
            continue
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        images_origin.append(image)  # keep for save result
        if len(images_origin) == 0:
            continue
        image = cv2.resize(image, (config["img_w"], config["img_h"]),
                            interpolation=cv2.INTER_LINEAR)
        image = image.astype(np.float32)
        image /= 255.0
        image = np.transpose(image, (2, 0, 1))
        image = image.astype(np.float32)
        images.append(image)
    images = np.asarray(images)
    images = torch.from_numpy(images).cuda()
    # inference
    with torch.no_grad():
        print(images.shape)
        outputs = detector(images)
        output_list = []
        for i in range(3):
            output_list.append(yolo_losses[i](outputs[i]))
        output = torch.cat(output_list, 1)
        batch_detections = non_max_suppression(output, config["yolo"]["classes"],
                                                conf_thres=0.5,
                                                nms_thres=0.45)

    # write result images. Draw bounding boxes and labels of detections
    classes = open('/content/MiDaS/YOLOv3_PyTorch/data/yolo_data/customdata/classes.txt', "r").read().split("\n")[:-1]
    if not os.path.isdir("./output/"):
        os.makedirs("./output/")
    for idx, detections in enumerate(batch_detections):
        # plt.figure()
        fig, ax = plt.subplots(1)
        ax.imshow(images_origin[idx])
        if detections is not None:
            unique_labels = detections[:, -1].cpu().unique()
            n_cls_preds = len(unique_labels)
            bbox_colors = random.sample(colors, n_cls_preds)
            for x1, y1, x2, y2, conf, cls_conf, cls_pred in detections:
                color = bbox_colors[int(np.where(unique_labels == int(cls_pred))[0])]
                # Rescale coordinates to original dimensions
                ori_h, ori_w = images_origin[idx].shape[:2]
                pre_h, pre_w = config["img_h"], config["img_w"]
                box_h = ((y2 - y1) / pre_h) * ori_h
                box_w = ((x2 - x1) / pre_w) * ori_w
                y1 = (y1 / pre_h) * ori_h
                x1 = (x1 / pre_w) * ori_w
                # Create a Rectangle patch
                bbox = patches.Rectangle((x1, y1), box_w, box_h, linewidth=2,
                                            edgecolor=color,
                                            facecolor='none')
                # Add the bbox to the plot
                ax.add_patch(bbox)
                # Add label
                print(int(cls_pred), len(classes), classes[int(cls_pred)])
                plt.text(x1, y1, s=classes[int(cls_pred)], color='white',
                            verticalalignment='top',
                            bbox={'color': color, 'pad': 0})
        # Save generated image with detections
        plt.axis('off')
        plt.gca().xaxis.set_major_locator(NullLocator())
        plt.gca().yaxis.set_major_locator(NullLocator())
        plt.savefig('output/{}_{}.jpg'.format(step, idx), bbox_inches='tight', pad_inches=0.0)
        plt.close()
logging.info("Save all results to ./output/")    


In [ ]:
# ! rm -rf /content/MiDaS/YOLOv3_PyTorch/common/data/

In [ ]:
# ! cp -r /content/drive/MyDrive/Annotated\ Images/labels/  /content/MiDaS/YOLOv3_PyTorch/data/custom/labels/
# ! cp -r /content/drive/MyDrive/Annotated\ Images/images/  /content/MiDaS/YOLOv3_PyTorch/data/custom/images/

In [ ]:
# ! cp /content/drive/MyDrive/ConstructionPPE/input_file_dir.txt  /content/MiDaS/YOLOv3_PyTorch/data/custom/